# Derivation of euler equations in primitive form
Derivation of the Weiss and Smith preconditioning.
We are following the 1D derivation of maxima_weiss_smith_1D here and extend it to 2D

In [1]:
kill(all);

(%o0)                                done

# Analysis of the Euler Equations
## 1. conservative form


    

We want to solve the PDE:

$$ \frac{\partial U}{\partial t} + \frac{\partial F(U)}{\partial x} + \frac{\partial G(U)}{\partial y} = 0$$


    
For the 2D problem we have the following definitions:


$$
U = \left\{
    \begin{array}\\
        \rho               &  \\
        \rho u        &  \\
        \rho E u & 
    \end{array}
\right\}
$$


$$
F = \left\{
    \begin{array}\\
        \rho  u                  &  \\
        p+\rho u^2               &  \\
        \rho  u  v          &  \\
        (\rho E + p) u & 
    \end{array}
\right\}
$$


$$
G = \left\{
    \begin{array}\\
        \rho  u  v          &  \\
        p+\rho v^2               &  \\
        (\rho E + p) u & 
    \end{array}
\right\}
$$



In [2]:
/*consSystem : [U1 = rho, U2 = rho*u, U3 = rho*E];*/
/*F:matrix([rho*u,rho*u^2+p,u*rho*E+p*u]);*/


consSystem : [U1=rho,U2=rho*u, U3=rho*v, U4=rho*E];

F:ratsimp((matrix([rho*u,rho*u*u+p,rho*u*v,u*rho*E+p*u])));
    
G:ratsimp((matrix([rho*v,rho*u*v,rho*v*v+p,v*rho*E+p*v])));

(%o1)           [U1 = rho, U2 = rho u, U3 = rho v, U4 = E rho]

(%o2)            [             2                             ]
                 [ rho u  rho u  + p  rho u v  (E rho + p) u ]

(%o3)            [                      2                    ]
                 [ rho v  rho u v  rho v  + p  (E rho + p) v ]

    
    We define the velocity q as follows:
    


In [3]:
q : (u*u+v*v)/2;

                                     2    2
                                    v  + u
(%o4)                               -------
                                       2

    The internal energy follows from the relationships : 
    
    $$E = H - \frac{p}{\rho} = e + q$$
    $$H = h + q = c_pT + q$$
    
    $E = c_pT + q - \frac{p}{\rho}$
    

In [4]:
e : E-q;

                                       2    2
                                      v  + u
(%o5)                             E - -------
                                         2

    
    The pressure follows from the pressure definition: $p = (\gamma - 1)\rho e$
    


In [5]:
p: (gamma-1)*rho*e;

                                   2    2
                                  v  + u
(%o6)                    rho (E - -------) (gamma - 1)
                                     2

    
We now map the conservative variables to $(\rho,u,v,E)$
    


In [6]:
mapping : [rho = U1, u=U2/U1, v = U3/U1, E=U4/U1];

                                     U2      U3      U4
(%o7)                 [rho = U1, u = --, v = --, E = --]
                                     U1      U1      U1

    
From this also follows the pressure in terms of the conservative variables:
    


In [7]:
P : subst(mapping,p);

                                   2     2
                                 U3    U2
                                 --- + ---
                                   2     2
                            U4   U1    U1
(%o8)                   U1 (-- - ---------) (gamma - 1)
                            U1       2

    
We now write the convective flux as function of the conservative variables:



In [8]:
Fstar : ratexpand(ratsimp(ev(subst(mapping,F),nouns)));
transpose(ratsimp(Fstar));

(%o9)  Col 1 = [ U2 ]
         [              2           2                2        2 ]
 Col 2 = [            U3  gamma   U2  gamma        U3     3 U2  ]
         [ U4 gamma - --------- - --------- - U4 + ---- + ----- ]
         [              2 U1        2 U1           2 U1   2 U1  ]
         [ U2 U3 ]
 Col 3 = [ ----- ]
         [  U1   ]
         [                    2           3              2      3  ]
         [ U2 U4 gamma   U2 U3  gamma   U2  gamma   U2 U3     U2   ]
 Col 4 = [ ----------- - ------------ - --------- + ------ + ----- ]
         [     U1               2             2         2        2 ]
         [                  2 U1          2 U1      2 U1     2 U1  ]

            [                         U2                          ]
            [                                                     ]
            [              2     2                      2       2 ]
            [ (2 U1 U4 - U3  - U2 ) gamma - 2 U1 U4 + U3  + 3 U2  ]
            [ --------------------------------------------------- ]
            [                        2 U1                         ]
            [                                                     ]
(%o10)      [                        U2 U3                        ]
            [                        -----                        ]
            [                         U1                          ]
            [                                                     ]
            [                     2     3               2     3   ]
            [  (2 U1 U2 U4 - U2 U3  - U2 ) gamma + U2 U3  + U2    ]
            [  ------------------------------------------------   ]
            [                           2       

In [9]:
Utilde:matrix([U1,U2,U3,U4]);

(%o11)                        [ U1  U2  U3  U4 ]

In [10]:
dFdU:ratsimp(jacobian(args(Fstar)[1],args(Utilde)[1]));

                [                        0                         ]
                [                                                  ]
                [            2     2            2       2          ]
                [         (U3  + U2 ) gamma - U3  - 3 U2           ]
                [         -------------------------------          ]
                [                          2                       ]
                [                      2 U1                        ]
                [                                                  ]
(%o12)  Col 1 = [                       U2 U3                      ]
                [                     - -----                      ]
                [                          2                       ]
                [                        U1                        ]
                [                                                  ]
                [                    2     3               2     3 ]
                [   (U1 U2 U4 - U2

    
Since $\frac{\partial F(U)}{\partial x} = \frac{\partial F}{\partial U}\frac{\partial U}{\partial x}$ we can now write 

the euler equation in quasilinear form with 
$$\frac{dU}{dt}+\frac{\partial F}{\partial U}\frac{dU}{dx}=\frac{dU}{dt}+A\frac{dU}{dx}=0$$
    


In [11]:
A1:ratsimp(subst(consSystem,dFdU));

                [                  0                  ]
                [                                     ]
                [       2    2           2      2     ]
                [     (v  + u ) gamma - v  - 3 u      ]
                [     ---------------------------     ]
(%o13)  Col 1 = [                  2                  ]
                [                                     ]
                [                - u v                ]
                [                                     ]
                [     2    3                   2    3 ]
                [ (u v  + u  - E u) gamma - u v  - u  ]
         [                   1                   ]
         [                                       ]         [        0        ]
         [             3 u - u gamma             ]         [                 ]
         [                                       ]         [   v - v gamma   ]
 Col 2 = [                   v                   ] Col 3 = [                 ]
         [       

    
We can get the internal energy from the pressure equation and write pressure as $p=\rho c^2/\gamma$:
    


In [12]:
pc : rho*c^2/gamma;
e : pc/(rho*(gamma-1));

                                     2
                                    c  rho
(%o14)                              ------
                                    gamma

                                       2
                                      c
(%o15)                         -----------------
                               (gamma - 1) gamma

In [13]:
A1 : ratsimp(subst([E = (e + q)],A1));

                               2    2           2      2
                             (v  + u ) gamma - v  - 3 u
(%o16) matrix([0, 1, 0, 0], [---------------------------, 3 u - u gamma, 
                                          2
v - v gamma, gamma - 1], [- u v, v, u, 0], 
     2    3       2            2       3               2      3      2
 (u v  + u ) gamma  + ((- 3 u v ) - 3 u ) gamma + 2 u v  + 2 u  - 2 c  u
[-----------------------------------------------------------------------, 
                               2 gamma - 2
     2      2        2       2           2      2      2
  2 u  gamma  + ((- v ) - 5 u ) gamma + v  + 3 u  - 2 c
- ------------------------------------------------------, u v - u v gamma, 
                       2 gamma - 2
u gamma])

    
    We can now compute the eigenvalues, which should be $(u, u, u-c, u+c)$
    


In [14]:
ratexpand(ratsimp(eigenvalues(A1)));

(%o17)                  [[u, u - c, u + c], [2, 1, 1]]

    
We use the same approach for the matrix B:



In [15]:
Gstar : ratexpand(ratsimp(ev(subst(mapping,G),nouns)));
dGdU:ratsimp(jacobian(args(Gstar)[1],args(Utilde)[1]));
B1:ratsimp(subst(consSystem,dGdU));
B1 : ratsimp(subst([E = (e + q)],B1));

                               [ U2 U3 ]
(%o18)  Col 1 = [ U3 ] Col 2 = [ ----- ]
                               [  U1   ]
         [              2           2                  2     2  ]
 Col 3 = [            U3  gamma   U2  gamma        3 U3    U2   ]
         [ U4 gamma - --------- - --------- - U4 + ----- + ---- ]
         [              2 U1        2 U1           2 U1    2 U1 ]
         [                 3           2               3      2    ]
         [ U3 U4 gamma   U3  gamma   U2  U3 gamma    U3     U2  U3 ]
 Col 4 = [ ----------- - --------- - ------------ + ----- + ------ ]
         [     U1              2            2           2       2  ]
         [                 2 U1         2 U1        2 U1    2 U1   ]

                [                        0                         ]
                [                                                  ]
                [                       U2 U3                      ]
                [                     - -----                      ]
                [                          2                       ]
                [                        U1                        ]
                [                                                  ]
                [            2     2              2     2          ]
(%o19)  Col 1 = [         (U3  + U2 ) gamma - 3 U3  - U2           ]
                [         -------------------------------          ]
                [                          2                       ]
                [                      2 U1                        ]
                [                                                  ]
                [                 3     2               3     2    ]
                [   (U1 U3 U4 - U3

                [                  0                  ]
                [                                     ]
                [                - u v                ]
                [                                     ]
                [       2    2             2    2     ]
(%o20)  Col 1 = [     (v  + u ) gamma - 3 v  - u      ]
                [     ---------------------------     ]
                [                  2                  ]
                [                                     ]
                [   3     2                  3    2   ]
                [ (v  + (u  - E) v) gamma - v  - u  v ]
                                     [                   1                   ]
         [        0        ]         [                                       ]
         [                 ]         [                   u                   ]
         [        v        ]         [                                       ]
 Col 2 = [                 ] Col 3 = [             3 v - v gamma    

(%o21) matrix([0, 0, 1, 0], [- u v, v, u, 0], 
   2    2             2    2
 (v  + u ) gamma - 3 v  - u
[---------------------------, u - u gamma, 3 v - v gamma, gamma - 1], 
              2
   3    2         2          3       2               3       2      2
 (v  + u  v) gamma  + ((- 3 v ) - 3 u  v) gamma + 2 v  + (2 u  - 2 c ) v
[-----------------------------------------------------------------------, 
                               2 gamma - 2
                      2      2          2     2             2    2      2
                   2 v  gamma  + ((- 5 v ) - u ) gamma + 3 v  + u  - 2 c
u v - u v gamma, - ------------------------------------------------------, 
                                        2 gamma - 2
v gamma])

    
We now have the PDE in quasi-linear form:

$$ \frac{\partial U}{\partial t} + A \cdot \frac{\partial U}{\partial x} + B \cdot \frac{\partial U}{\partial y} = 0$$
    



***
    We can now compute the eigenvalues, which should be $(u, u, u-c, u+c)$  

***

In [16]:
ratexpand(ratsimp(eigenvalues(A1)));

(%o22)                  [[u, u - c, u + c], [2, 1, 1]]

In [17]:
ratexpand(ratsimp(eigenvalues(B1)));

(%o23)                  [[v, v - c, v + c], [2, 1, 1]]

# 2. primitive form with $V=(\rho,u,v,P)$

The euler equation is now in quasilinear form with 

$$ \frac{\partial U}{\partial t} + A \cdot \frac{\partial U}{\partial x} + B \cdot \frac{\partial U}{\partial y} = 0$$
    
We now write the equations in terms of the new primitive variables $V=(\rho,u,v,p)$.


    
We have the relationships:
    
    
    
    $$\frac{\partial U(V)}{\partial t} = \frac{\partial U}{\partial V}\frac{\partial V}{\partial t} = M \frac{\partial V}{\partial t}$$
    
    
    $$\frac{\partial U(V)}{\partial x} = \frac{\partial U}{\partial V}\frac{\partial V}{\partial x} = M \frac{\partial V}{\partial x}$$
    
the Euler equations in quasilinear form can now be written as: 
$$\frac{dU(V)}{dt} + A\frac{dU(V)}{dx}=M\frac{\partial V}{\partial t} + A M \frac{\partial V}{\partial x}=0$$
    
Or, when collecting the matrices:
    
$$\frac{\partial V}{\partial t} + M^{-1} A M \frac{\partial V}{\partial x}=0$$

    



In [18]:
/* compact representation of primitive variables rho, u, p*/

RHO : U1;
Up : U2/U1;
Vp : U3/U1;
P;

(%o24)                                U1

                                      U2
(%o25)                                --
                                      U1

                                      U3
(%o26)                                --
                                      U1

                                   2     2
                                 U3    U2
                                 --- + ---
                                   2     2
                            U4   U1    U1
(%o27)                  U1 (-- - ---------) (gamma - 1)
                            U1       2

In [19]:
Vtilde:ratsimp(matrix([RHO,Up,Vp, P]));

       [                          2     2                      2     2 ]
(%o28) [     U2  U3  (2 U1 U4 - U3  - U2 ) gamma - 2 U1 U4 + U3  + U2  ]
       [ U1  --  --  ------------------------------------------------- ]
       [     U1  U1                        2 U1                        ]

In [20]:
Utilde:matrix([U1,U2,U3,U4]);
dVdU:ratsimp(jacobian(args(Vtilde)[1],args(Utilde)[1]));
dUdV : ratsimp(invert(dVdU));
M : ratsimp(subst(consSystem,dUdV));
ratsimp(invert(M));
Atilde : ratsimp(invert(M).A1.M);
ratexpand(ratsimp(eigenvalues(Atilde)));

(%o29)                        [ U1  U2  U3  U4 ]

(%o30) 
 [               1                       0                0             0     ]
 [                                                                            ]
 [               U2                     1                                     ]
 [             - ---                    --                0             0     ]
 [                 2                    U1                                    ]
 [               U1                                                           ]
 [                                                                            ]
 [               U3                                      1                    ]
 [             - ---                     0               --             0     ]
 [                 2                                     U1                   ]
 [               U1                                                           ]
 [                                                                            ]
 [    2     2            2     2

                       [     1      0   0       0     ]
                       [                              ]
                       [    U2                        ]
                       [    --      U1  0       0     ]
                       [    U1                        ]
                       [                              ]
                       [    U3                        ]
(%o31)                 [    --      0   U1      0     ]
                       [    U1                        ]
                       [                              ]
                       [   2     2                    ]
                       [ U3  + U2               1     ]
                       [ ---------  U2  U3  --------- ]
                       [       2            gamma - 1 ]
                       [   2 U1                       ]

                     [    1       0      0        0     ]
                     [                                  ]
                     [    u      rho     0        0     ]
                     [                                  ]
(%o32)               [    v       0     rho       0     ]
                     [                                  ]
                     [  2    2                          ]
                     [ v  + u                     1     ]
                     [ -------  rho u  rho v  --------- ]
                     [    2                   gamma - 1 ]

       [             1                   0            0           0     ]
       [                                                                ]
       [              u                  1                              ]
       [           - ---                ---           0           0     ]
       [             rho                rho                             ]
       [                                                                ]
(%o33) [              v                               1                 ]
       [           - ---                 0           ---          0     ]
       [             rho                             rho                ]
       [                                                                ]
       [   2    2           2    2                                      ]
       [ (v  + u ) gamma - v  - u                                       ]
       [ -------------------------  u - u gamma  v - v gamma  gamma - 1 ]
       [             2                

                             [ u   rho    0   0  ]
                             [                   ]
                             [                1  ]
                             [ 0    u     0  --- ]
(%o34)                       [               rho ]
                             [                   ]
                             [ 0    0     u   0  ]
                             [                   ]
                             [     2             ]
                             [ 0  c  rho  0   u  ]

(%o35)                  [[u - c, u + c, u], [1, 1, 2]]

Note that this corresponds (besides some typing errors) to the matrices in the lecture notes of Jameson AA215A, lecture 4, p 10

# 3. primitive form with $V=(P,u,v,T)$



We have 

$$E = c_pT + q - \frac{p}{\rho}$$

$$ T = \frac{\rho E - \rho q + p }{\rho c_p}$$

Now write $T$ in terms of the variables $U_1=\rho$, $U_2=\rho u$, $U_3 = \rho v$, $U_4 = \rho E$.
We already have P in terms of U


In [21]:
Q : ((U2/U1)^2 + (U3/U1)^2)/2;
T : ratsimp((U4 - U1*Q + P)/(U1*cp));

                                     2     2
                                   U3    U2
                                   --- + ---
                                     2     2
                                   U1    U1
(%o36)                             ---------
                                       2

                                       2     2
                          (2 U1 U4 - U3  - U2 ) gamma
(%o37)                    ---------------------------
                                       2
                                   2 U1  cp

In [22]:
/* this is the only thing that needs to be changed to switch to other primitive variables*/
Vtilde:ratsimp(matrix([P,Up,Vp,T]));

                [              2     2                      2     2 ]
(%o38)  Col 1 = [ (2 U1 U4 - U3  - U2 ) gamma - 2 U1 U4 + U3  + U2  ]
                [ ------------------------------------------------- ]
                [                       2 U1                        ]
                                                [              2     2        ]
                  [ U2 ]         [ U3 ]         [ (2 U1 U4 - U3  - U2 ) gamma ]
          Col 2 = [ -- ] Col 3 = [ -- ] Col 4 = [ --------------------------- ]
                  [ U1 ]         [ U1 ]         [              2              ]
                                                [          2 U1  cp           ]

In [23]:
Utilde:matrix([U1,U2,U3,U4]);
dVdU:ratsimp(jacobian(args(Vtilde)[1],args(Utilde)[1]));
dUdV : ratsimp(invert(dVdU));
consSystem;
M : ratsimp(subst(consSystem,dUdV));
/* substitute energy*/
M : ratsimp(subst(E=e+q,M));
A1;
Atilde : ratsimp(invert(M).A1.M);
ratexpand(ratsimp(eigenvalues(Atilde)));

(%o39)                        [ U1  U2  U3  U4 ]

(%o40) 
 [    2     2            2     2                                              ]
 [ (U3  + U2 ) gamma - U3  - U2     U2 gamma - U2    U3 gamma - U3            ]
 [ -----------------------------  - -------------  - -------------  gamma - 1 ]
 [                 2                     U1               U1                  ]
 [             2 U1                                                           ]
 [                                                                            ]
 [               U2                     1                                     ]
 [             - ---                    --                0             0     ]
 [                 2                    U1                                    ]
 [               U1                                                           ]
 [                                                                            ]
 [               U3                                      1                    ]
 [             - ---            

                [                           2                       ]
                [                       2 U1                        ]
                [ ------------------------------------------------- ]
                [              2     2                      2     2 ]
                [ (2 U1 U4 - U3  - U2 ) gamma - 2 U1 U4 + U3  + U2  ]
                [                                                   ]
                [                      2 U1 U2                      ]
                [ ------------------------------------------------- ]
                [              2     2                      2     2 ]
(%o41)  Col 1 = [ (2 U1 U4 - U3  - U2 ) gamma - 2 U1 U4 + U3  + U2  ]
                [                                                   ]
                [                      2 U1 U3                      ]
                [ ------------------------------------------------- ]
                [              2     2                      2     2 ]
                [ (2

(%o42)          [U1 = rho, U2 = rho u, U3 = rho v, U4 = E rho]

                [                     2                   ]
                [ - ------------------------------------- ]
                [     2    2                 2    2       ]
                [   (v  + u  - 2 E) gamma - v  - u  + 2 E ]
                [                                         ]
                [                    2 u                  ]
                [ - ------------------------------------- ]         [   0   ]
                [     2    2                 2    2       ]         [       ]
                [   (v  + u  - 2 E) gamma - v  - u  + 2 E ]         [  rho  ]
(%o43)  Col 1 = [                                         ] Col 2 = [       ]
                [                    2 v                  ]         [   0   ]
                [ - ------------------------------------- ]         [       ]
                [     2    2                 2    2       ]         [ rho u ]
                [   (v  + u  - 2 E) gamma - v  - u  + 2 E ]
                [                 

                [                     gamma                     ]
                [                     -----                     ]
                [                       2                       ]
                [                      c                        ]
                [                                               ]
                [                    u gamma                    ]
                [                    -------                    ]
                [                       2                       ]
                [                      c                        ]
(%o44)  Col 1 = [                                               ]
                [                    v gamma                    ]
                [                    -------                    ]
                [                       2                       ]
                [                      c                        ]
                [                                               ]
          

                               2    2           2      2
                             (v  + u ) gamma - v  - 3 u
(%o45) matrix([0, 1, 0, 0], [---------------------------, 3 u - u gamma, 
                                          2
v - v gamma, gamma - 1], [- u v, v, u, 0], 
     2    3       2            2       3               2      3      2
 (u v  + u ) gamma  + ((- 3 u v ) - 3 u ) gamma + 2 u v  + 2 u  - 2 c  u
[-----------------------------------------------------------------------, 
                               2 gamma - 2
     2      2        2       2           2      2      2
  2 u  gamma  + ((- v ) - 5 u ) gamma + v  + 3 u  - 2 c
- ------------------------------------------------------, u v - u v gamma, 
                       2 gamma - 2
u gamma])

                             [       2           ]
                             [  u   c  rho  0  0 ]
                             [                   ]
                             [  1                ]
                             [ ---    u     0  0 ]
                             [ rho               ]
(%o46)                       [                   ]
                             [  0     0     u  0 ]
                             [                   ]
                             [         2         ]
                             [        c          ]
                             [  0     --    0  u ]
                             [        cp         ]

(%o47)                  [[u - c, u + c, u], [1, 1, 2]]


Note that the PDE is now written in terms of primitive variables as:

$$\frac{\partial V}{\partial t} + M^{-1} A M \frac{\partial V}{\partial x}=0$$
$$\frac{\partial V}{\partial t} + \tilde A \frac{\partial V}{\partial x}=0$$
    


## 3.1 preconditioning
And we can use a preconditioner matrix in front of the temporal derivative to influence the eigenvalues. The steady state solution will still be the same.
    


In [24]:
Gamma : matrix([theta,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]);

                              [ theta  0  0  0 ]
                              [                ]
                              [   0    1  0  0 ]
(%o48)                        [                ]
                              [   0    0  1  0 ]
                              [                ]
                              [   0    0  0  1 ]

In [25]:
invert(Gamma).Atilde;

                            [         2           ]
                            [   u    c  rho       ]
                            [ -----  ------  0  0 ]
                            [ theta  theta        ]
                            [                     ]
                            [   1                 ]
                            [  ---     u     0  0 ]
(%o49)                      [  rho                ]
                            [                     ]
                            [   0      0     u  0 ]
                            [                     ]
                            [           2         ]
                            [          c          ]
                            [   0      --    0  u ]
                            [          cp         ]

In [26]:
lambdas: eigenvalues(invert(Gamma).Atilde);

                      2                 2      2
           sqrt((theta  - 2 theta + 1) u  + 4 c  theta) + ((- theta) - 1) u
(%o50) [[- ----------------------------------------------------------------, 
                                       2 theta
              2                 2      2
   sqrt((theta  - 2 theta + 1) u  + 4 c  theta) + (theta + 1) u
   ------------------------------------------------------------, u], [1, 1, 2]]
                             2 theta

In [27]:
ratsimp(lambdas[1][1]/lambdas[1][2]);

                    2                 2      2
         sqrt((theta  - 2 theta + 1) u  + 4 c  theta) + ((- theta) - 1) u
(%o51) - ----------------------------------------------------------------
                      2                 2      2
           sqrt((theta  - 2 theta + 1) u  + 4 c  theta) + (theta + 1) u

# 4. Preconditioning matrix $\Gamma$

The above shows how the conversion from conservative to primitive variables work. Note that in the paper of Weiss and Smith, the derivatives of density with respect to temperature and pressure are kept.

Note that $P=\rho R T$, and $\rho=\frac{P}{RT}$, so the derivative is:

$$ \frac{\partial \rho}{\partial p} = \frac{1}{RT} $$

And since $c^2 = \gamma R T$, we have:

$$ \frac{\partial \rho}{\partial p} = \frac{\gamma}{c^2} $$

For derivative wrt temperarure we have :
    


In [28]:
kill(T);
kill(p);

subst(p=rho*R*T,diff(p/(R*T),T));

(%o52)                               done

(%o53)                               done

                                       rho
(%o54)                               - ---
                                        T


$$ \frac{\partial \rho}{\partial T} = -\frac{\rho}{T} $$

We can substitute these into the matrix M above to get the result from Weiss and Smith:
    


In [29]:
depends(rho,[p,T]);

(%o55)                            [rho(p, T)]


Now we compute the transformation matrix $M = \frac{\partial U}{\partial V}$, equation (7) in the paper of Economon, or below eq. 2 in Weiss and Smith:


In [30]:

V:matrix([p,u,v,T]);

E:H - p/rho;

gradef(H,T,cp);
gradef(H,u,u);
gradef(H,v,v);
gradef(H,w,w);

U:matrix([rho,rho*u,rho*v,rho*E]);
V;
/* *we directly convert from conservative variables to primitive variables */
dWdQ : ratsimp(jacobian(args(U)[1],args(V)[1]));

(%o56)                          [ p  u  v  T ]

                                         p
(%o57)                              H - ---
                                        rho

(%o58)                                 H

(%o59)                                 H

(%o60)                                 H

(%o61)                                 H

                     [                          p       ]
(%o62)               [ rho  rho u  rho v  (H - ---) rho ]
                     [                         rho      ]

(%o63)                          [ p  u  v  T ]

                 [    drho                        drho       ]
                 [    ----       0      0         ----       ]
                 [     dp                          dT        ]
                 [                                           ]
                 [   drho                        drho        ]
                 [   ---- u     rho     0        ---- u      ]
                 [    dp                          dT         ]
(%o64)           [                                           ]
                 [   drho                        drho        ]
                 [   ---- v      0     rho       ---- v      ]
                 [    dp                          dT         ]
                 [                                           ]
                 [   drho                      drho          ]
                 [ H ---- - 1  rho u  rho v  H ---- + cp rho ]
                 [    dp                        dT           ]

We see that we have recovered the result of Weiss and Smith, matrix $\frac{\partial W}{\partial Q}$, below equation (2). Note that this should be the same as the matrix M defined above:

In [31]:
M;

                [                     gamma                     ]
                [                     -----                     ]
                [                       2                       ]
                [                      c                        ]
                [                                               ]
                [                    u gamma                    ]
                [                    -------                    ]
                [                       2                       ]
                [                      c                        ]
(%o65)  Col 1 = [                                               ]
                [                    v gamma                    ]
                [                    -------                    ]
                [                       2                       ]
                [                      c                        ]
                [                                               ]
          

Note that $p=\rho R T$, and $\frac{a^2}{\gamma} = \frac{p}{\rho}$, so:
$$RT = \frac{a^2}{\gamma}$$

and with $\frac{c_p}{c_v}=\gamma$ and  $R = c_p - c_v = c_p(1-\frac{1}{\gamma})$:
$$T = \frac{a^2}{\gamma}\frac{1}{c_p(1-\frac{1}{\gamma})},$$
and finally:

$$ T = \frac{a^2}{ c_p (\gamma - 1)}$$

In [32]:
dWdQ2 : subst([diff(rho,T)=-rho/T,diff(rho,p)=gamma/c^2,H=cp*T+q],dWdQ);
dWdQ2 : ratsimp(subst(T=c*c/(cp*(gamma-1)),dWdQ2));

(%o66) 
    [           gamma                                         rho             ]
    [           -----               0      0                - ---             ]
    [             2                                            T              ]
    [            c                                                            ]
    [                                                                         ]
    [          u gamma                                       rho u            ]
    [          -------             rho     0               - -----            ]
    [             2                                            T              ]
    [            c                                                            ]
    [                                                                         ]
    [          v gamma                                       rho v            ]
    [          -------              0     rho              - -----            ]
    [             2             

                [                     gamma                     ]
                [                     -----                     ]
                [                       2                       ]
                [                      c                        ]
                [                                               ]
                [                    u gamma                    ]
                [                    -------                    ]
                [                       2                       ]
                [                      c                        ]
(%o67)  Col 1 = [                                               ]
                [                    v gamma                    ]
                [                    -------                    ]
                [                       2                       ]
                [                      c                        ]
                [                                               ]
          

Notice that using the two different routes, we obtain the same result, as $\frac{\partial W}{\partial Q} = M$

In [33]:
dWdQ;

                 [    drho                        drho       ]
                 [    ----       0      0         ----       ]
                 [     dp                          dT        ]
                 [                                           ]
                 [   drho                        drho        ]
                 [   ---- u     rho     0        ---- u      ]
                 [    dp                          dT         ]
(%o68)           [                                           ]
                 [   drho                        drho        ]
                 [   ---- v      0     rho       ---- v      ]
                 [    dp                          dT         ]
                 [                                           ]
                 [   drho                      drho          ]
                 [ H ---- - 1  rho u  rho v  H ---- + cp rho ]
                 [    dp                        dT           ]

Note that the flux is $\frac{\partial F(V)}{\partial x}$, with $V=(p,u,v,T)$ and because we have defined the derivative of density with respect to pressure and temperarure, we can obtain the transformation into quasilinear form directly using:
$$\frac{\partial F(V)}{\partial x} = \frac{\partial F}{\partial V}\frac{\partial V}{\partial x}$$

In [34]:

F:ratsimp((matrix([rho*u,rho*u*u+p,rho*u*v,u*rho*E+p*u])));

(%o69)              [             2                       ]
                    [ rho u  rho u  + p  rho u v  H rho u ]

and its Jacobian with respect to $V=(\rho, u,v, T)$ can be determined directly (because we also have the derivative of H(T) defined):

In [35]:
V;

(%o70)                          [ p  u  v  T ]

In [36]:
dFdV : jacobian(args(F)[1],args(V)[1]);

         [   drho                                      drho          ]
         [   ---- u          rho           0           ---- u        ]
         [    dp                                        dT           ]
         [                                                           ]
         [ drho  2                                     drho  2       ]
         [ ---- u  + 1     2 rho u         0           ---- u        ]
         [  dp                                          dT           ]
(%o71)   [                                                           ]
         [  drho                                      drho           ]
         [  ---- u v        rho v        rho u        ---- u v       ]
         [   dp                                        dT            ]
         [                                                           ]
         [    drho           2                     drho              ]
         [  H ---- u    rho u  + H rho  rho u v  H ---- u + cp rho u ]
      

In [37]:
dFdV2 : subst([diff(rho,T)=-rho/T,diff(rho,p)=gamma/c^2,H=cp*T+q],dFdV);
dFdV2 : ratsimp(subst(T=c*c/(cp*(gamma-1)),dFdV2));

                [         u gamma          ]
                [         -------          ]
                [            2             ]
                [           c              ]
                [                          ]
                [        2                 ]
                [       u  gamma           ]
                [       -------- + 1       ]
                [           2              ]
                [          c               ]
                [                          ]
(%o72)  Col 1 = [        u v gamma         ]
                [        ---------         ]
                [            2             ]
                [           c              ]
                [                          ]
                [     2    2               ]
                [    v  + u                ]
                [ u (------- + T cp) gamma ]
                [       2                  ]
                [ ------------------------ ]
                [             2            ]
          

                [                       u gamma                       ]
                [                       -------                       ]
                [                          2                          ]
                [                         c                           ]
                [                                                     ]
                [                     2          2                    ]
                [                    u  gamma + c                     ]
                [                    -------------                    ]
                [                          2                          ]
                [                         c                           ]
(%o73)  Col 1 = [                                                     ]
                [                      u v gamma                      ]
                [                      ---------                      ]
                [                          2                    

We can now write the pde in primitive variables $V$ as:
$$\frac{\partial U}{\partial t} + \frac{\partial F}{\partial x}=0$$
$$\frac{\partial U}{\partial V} \frac{\partial V}{\partial t} + \frac{\partial F}{\partial V}\frac{\partial V}{\partial x}=0$$
$$ \frac{\partial V}{\partial t} + [\frac{\partial U}{\partial V}]^{-1} \frac{\partial F}{\partial V}\frac{\partial V}{\partial x}=0$$


In [38]:
ratsimp(invert(dWdQ2));

                [             2    2           2    2           ]
                [           (v  + u ) gamma - v  - u            ]
                [           -------------------------           ]
                [                       2                       ]
                [                                               ]
                [                        u                      ]
                [                     - ---                     ]
                [                       rho                     ]
(%o74)  Col 1 = [                                               ]
                [                        v                      ]
                [                     - ---                     ]
                [                       rho                     ]
                [                                               ]
                [   2    2       2        2     2             2 ]
                [ (v  + u ) gamma  + ((- v ) - u ) gamma - 2 c  ]
          

In [39]:
ratsimp(invert(dWdQ2).dFdV2);

                             [       2           ]
                             [  u   c  rho  0  0 ]
                             [                   ]
                             [  1                ]
                             [ ---    u     0  0 ]
                             [ rho               ]
(%o75)                       [                   ]
                             [  0     0     u  0 ]
                             [                   ]
                             [         2         ]
                             [        c          ]
                             [  0     --    0  u ]
                             [        cp         ]

In [40]:
eigenvalues(ratsimp(invert(dWdQ2).dFdV2));

(%o76)                  [[u - c, u + c, u], [1, 1, 2]]

...And we see again that we retrieve the same matrix...
We now introduce a preconditioning matrix and see whether we can modify the eigenvalues in a beneficial way:
We use the Weiss and Smith approach that $\frac{\partial \rho}{\partial p} = \beta$

In [41]:
/* note that it is 2*q */
K:matrix([1,0,0,0],[-u,1,0,0],[-v,0,1,0],[-(H-2*q),-u,-v,1]);
K2 : subst([diff(rho,T)=-rho/T,diff(rho,p)=gamma/c^2,H=cp*T+q],K);
K2 : ratsimp(subst(T=c*c/(cp*(gamma-1)),K2));

                         [      1        0    0   0 ]
                         [                          ]
                         [     - u       1    0   0 ]
(%o77)                   [                          ]
                         [     - v       0    1   0 ]
                         [                          ]
                         [  2    2                  ]
                         [ v  + u  - H  - u  - v  1 ]

                 [              1                 0    0   0 ]
                 [                                           ]
                 [             - u                1    0   0 ]
                 [                                           ]
(%o78)           [             - v                0    1   0 ]
                 [                                           ]
                 [     2    2                                ]
                 [    v  + u      2    2                     ]
                 [ (- -------) + v  + u  - T cp  - u  - v  1 ]
                 [       2                                   ]

               [                1                   0    0   0 ]
               [                                               ]
               [               - u                  1    0   0 ]
               [                                               ]
(%o79)         [               - v                  0    1   0 ]
               [                                               ]
               [   2    2           2    2      2              ]
               [ (v  + u ) gamma - v  - u  - 2 c               ]
               [ --------------------------------  - u  - v  1 ]
               [           2 gamma - 2                         ]

In [42]:
dWdQ;

                 [    drho                        drho       ]
                 [    ----       0      0         ----       ]
                 [     dp                          dT        ]
                 [                                           ]
                 [   drho                        drho        ]
                 [   ---- u     rho     0        ---- u      ]
                 [    dp                          dT         ]
(%o80)           [                                           ]
                 [   drho                        drho        ]
                 [   ---- v      0     rho       ---- v      ]
                 [    dp                          dT         ]
                 [                                           ]
                 [   drho                      drho          ]
                 [ H ---- - 1  rho u  rho v  H ---- + cp rho ]
                 [    dp                        dT           ]

In [43]:
ratsimp(K.dWdQ);

                          [ drho             drho  ]
                          [ ----   0    0    ----  ]
                          [  dp               dT   ]
                          [                        ]
(%o81)                    [  0    rho   0     0    ]
                          [                        ]
                          [  0     0   rho    0    ]
                          [                        ]
                          [ - 1    0    0   cp rho ]

In [44]:
eigenvalues(ratsimp(K.(ratsimp(invert(dWdQ2).dFdV2))));

                    2    2           2          2
           c sqrt((c  rho  - 4 rho) u  + 4) + (c  rho - 2) u
(%o82) [[- -------------------------------------------------, 
                                   2
                       2    2           2              2
              c sqrt((c  rho  - 4 rho) u  + 4) + (2 - c  rho) u
              -------------------------------------------------, u], [1, 1, 2]]
                                      2

In [45]:
Gamma : subst(diff(rho,p)=theta,dWdQ);

                [                                 drho       ]
                [    theta       0      0         ----       ]
                [                                  dT        ]
                [                                            ]
                [                                drho        ]
                [   theta u     rho     0        ---- u      ]
                [                                 dT         ]
(%o83)          [                                            ]
                [                                drho        ]
                [   theta v      0     rho       ---- v      ]
                [                                 dT         ]
                [                                            ]
                [                              drho          ]
                [ H theta - 1  rho u  rho v  H ---- + cp rho ]
                [                               dT           ]

This is the convective matrix for the quasilinear system with primitive variables $V$.

In [46]:
inv_dUdV : ratsimp(invert(M));

                [             2    2           2    2           ]
                [           (v  + u ) gamma - v  - u            ]
                [           -------------------------           ]
                [                       2                       ]
                [                                               ]
                [                        u                      ]
                [                     - ---                     ]
                [                       rho                     ]
(%o84)  Col 1 = [                                               ]
                [                        v                      ]
                [                     - ---                     ]
                [                       rho                     ]
                [                                               ]
                [   2    2       2        2     2             2 ]
                [ (v  + u ) gamma  + ((- v ) - u ) gamma - 2 c  ]
          

In [47]:
A2:ratsimp(inv_dUdV.dFdV);

                   drho    2   drho  3            drho             drho    2
(%o85) matrix([- ((---- u v  + ---- u  + (2 - 2 H ----) u) gamma - ---- u v
                    dp          dp                 dp               dp
   drho  3        drho
 - ---- u  + (2 H ---- - 2) u)/2, 
    dp             dp
        2        2                         2        2
  (rho v  + rho u  - 2 H rho) gamma - rho v  - rho u  + 2 H rho
- -------------------------------------------------------------, 0, 
                                2
    drho    2   drho  3           drho                         drho    2
- ((---- u v  + ---- u  + ((- 2 H ----) - 2 cp rho) u) gamma - ---- u v
     dT          dT                dT                           dT
   drho  3        drho                      1
 - ---- u  + (2 H ---- + 2 cp rho) u)/2], [---, u, 0, 0], [0, 0, u, 0], 
    dT             dT                      rho
     drho    2   drho  3            drho          2
[- ((---- u v  + ---- u  + (2 - 2 H ----) u) 

In [48]:
A3:ratsimp(subst(diff(rho,T)=-rho/T,A2));
A3:ratsimp(subst(diff(rho,p)=gamma/c^2,A3));

                   drho    2   drho  3            drho             drho    2
(%o86) matrix([- ((---- u v  + ---- u  + (2 - 2 H ----) u) gamma - ---- u v
                    dp          dp                 dp               dp
   drho  3        drho
 - ---- u  + (2 H ---- - 2) u)/2, 
    dp             dp
        2        2                         2        2
  (rho v  + rho u  - 2 H rho) gamma - rho v  - rho u  + 2 H rho
- -------------------------------------------------------------, 0, 
                                2
         2        3                                        2        3
((rho u v  + rho u  + (2 T cp - 2 H) rho u) gamma - rho u v  - rho u
                                   1
 + (2 H - 2 T cp) rho u)/(2 T)], [---, u, 0, 0], [0, 0, u, 0], 
                                  rho
     drho    2   drho  3            drho          2
[- ((---- u v  + ---- u  + (2 - 2 H ----) u) gamma
      dp          dp                 dp
       drho    2    drho  3        drho               

                      2    3               2
(%o87) matrix([- ((u v  + u  - 2 H u) gamma
          2     3       2                      2        2
 + ((- u v ) - u  + (2 c  + 2 H) u) gamma - 2 c  u)/(2 c ), 
        2        2                         2        2
  (rho v  + rho u  - 2 H rho) gamma - rho v  - rho u  + 2 H rho
- -------------------------------------------------------------, 0, 
                                2
         2        3                                        2        3
((rho u v  + rho u  + (2 T cp - 2 H) rho u) gamma - rho u v  - rho u
                                   1
 + (2 H - 2 T cp) rho u)/(2 T)], [---, u, 0, 0], [0, 0, u, 0], 
                                  rho
       2    3               3          2     3       2                2
   (u v  + u  - 2 H u) gamma  + ((- u v ) - u  + (2 c  + 2 H) u) gamma
[- --------------------------------------------------------------------, 
                        2                   2
                     2 c  cp r

We also see that the matrix $\tilde A$ actually should also depend on the derivatives of the density:

In [49]:
Atilde;

                             [       2           ]
                             [  u   c  rho  0  0 ]
                             [                   ]
                             [  1                ]
                             [ ---    u     0  0 ]
                             [ rho               ]
(%o88)                       [                   ]
                             [  0     0     u  0 ]
                             [                   ]
                             [         2         ]
                             [        c          ]
                             [  0     --    0  u ]
                             [        cp         ]

The matrix M is the transformation matrix to go from conservative to primitive variables. We should now also go from conservative form to quasilinear (nonconservative) form. For this, we need to write F in terms of the primitive variables and find $A = \frac{\partial F}{\partial U}$

First, F in terms of the primitive variables:

In [50]:
Atilde2 : ratsimp(invert(M).A1.M);

                             [       2           ]
                             [  u   c  rho  0  0 ]
                             [                   ]
                             [  1                ]
                             [ ---    u     0  0 ]
                             [ rho               ]
(%o89)                       [                   ]
                             [  0     0     u  0 ]
                             [                   ]
                             [         2         ]
                             [        c          ]
                             [  0     --    0  u ]
                             [        cp         ]

with this form, we can still retrieve the eigenvalues in their most simplified form:

In [51]:
ratexpand(ratsimp(eigenvalues(Atilde2)));

(%o90)                  [[u - c, u + c, u], [1, 1, 2]]

Note that we can now get the preconditioning matrix by simply replacing $\frac{\partial \rho}{\partial p}$ by the preconditioning constant $\beta$. This is because this is the only term that determines the eigenvalues $u+c$ and $u-c$. So if we want to modify the condition number, we need to modify this term.


In [52]:
Mp : subst(diff(rho,p)=beta,M);

                [                     gamma                     ]
                [                     -----                     ]
                [                       2                       ]
                [                      c                        ]
                [                                               ]
                [                    u gamma                    ]
                [                    -------                    ]
                [                       2                       ]
                [                      c                        ]
(%o91)  Col 1 = [                                               ]
                [                    v gamma                    ]
                [                    -------                    ]
                [                       2                       ]
                [                      c                        ]
                [                                               ]
          

In [53]:
Mp : subst(diff(rho,T)=-rho/T,Mp);

                [                     gamma                     ]
                [                     -----                     ]
                [                       2                       ]
                [                      c                        ]
                [                                               ]
                [                    u gamma                    ]
                [                    -------                    ]
                [                       2                       ]
                [                      c                        ]
(%o92)  Col 1 = [                                               ]
                [                    v gamma                    ]
                [                    -------                    ]
                [                       2                       ]
                [                      c                        ]
                [                                               ]
          

In [54]:
Mp : subst(H = cp*T +q,Mp);

                [                     gamma                     ]
                [                     -----                     ]
                [                       2                       ]
                [                      c                        ]
                [                                               ]
                [                    u gamma                    ]
                [                    -------                    ]
                [                       2                       ]
                [                      c                        ]
(%o93)  Col 1 = [                                               ]
                [                    v gamma                    ]
                [                    -------                    ]
                [                       2                       ]
                [                      c                        ]
                [                                               ]
          

What are the eigenvalues of the new system:

In [ ]:
Ap : ratsimp(invert(Mp).Atilde);
ratexpand(ratsimp(eigenvalues(Ap)));

                       2        3                       2        3
               (rho u v  + rho u  - 2 u) gamma - rho u v  - rho u  + 2 u
(%o94) matrix([---------------------------------------------------------, 
                                         2 rho
   2         2     2                 2      2           2         2
((c  cp rho v  + (c  cp rho - 2 cp) u  + 2 c ) gamma - c  cp rho v
            2          2      2
 + (2 cp - c  cp rho) u  - 2 c )/(2 cp), u v - u v gamma, u gamma - u], 
        2          2
   rho u  - 1    (c  rho - 1) u            u v     2     u
[- ----------, - --------------, 0, 0], [- ---, - c  v, ---, 0], 
         2            rho                  rho          rho
      rho
          2        3             2              2         3
[((rho u v  + rho u  - 2 u) gamma  + ((- rho u v ) - rho u  + 2 u) gamma
      2                 2                 2
 - 2 c  rho u)/(2 cp rho  gamma - 2 cp rho ), 
   2         2     2                 2      2       2
((c 

Note that in Weiss and Smith, 

For the matrix K we still need to derive it, for now we just give the matrix:

In [ ]:
K:matrix([1,0,0,0],[-u,1,0,0],[-v,0,1,0],[-cp*T,0,0,1]);

In [ ]:
invert(K);

In [ ]:
Gnc:matrix([1/b^2,0,0,diff(rho,T)],[0,rho,0,0],[0,0,rho,0],[0,0,0,rho*cp]);

In [ ]:
invert(Gnc);

In [ ]:
eigenvalues(invert(Gnc));

In [ ]:
Atilde;

preconditioning matrix, which can then be derived from K.M:
IN the end, this is the matrix in fron of the temporal termf of the equations in conservation form with primitive variables:

In [ ]:
G:matrix([1/b^2,0,0,diff(rho,T)],[u/b^2,rho,0,diff(rho,T)*u],[v/b^2,0,rho,diff(rho,T)*v],[cp*T/b^2,0,0,rho*cp+diff(rho,T)*cp*T]);

In [ ]:
Ginv : ratsimp(invert(G));

In [ ]:
L:eigenvalues(subst(diff(rho,T)=1/(R*T),Ginv));
ratsimp(L);

In [ ]:
(ratsimp(Ginv.Atilde));

note that $p=(\gamma-1)*(E-\frac{1}{2}\rho u^2)$

In [ ]:
kill(E);
Ftilde : subst([E=p/(gamma-1)+rho*u*u/2],F);
Ftilde: transpose(subst([rho=U1,u=U2,p=U3],Ftilde));

In [ ]:
Utilde;

In [ ]:
Vtilde;

In [ ]:
jacobian(args(Ftilde)[1],args(Utilde)[1]);

We now try to go to directly to primitive variables (p,u,T)

In [ ]:
F;
Ftilde : subst([E=rho*H-p],F);
Ftilde : transpose(subst(H=cp*T+u*u/2,Ftilde));

now replace all densities with pressures:

In [ ]:
Ftilde : subst(rho=gamma*p/(c*c),Ftilde);

In [ ]:
V;

In [ ]:
dfdv : (jacobian(args(Ftilde)[1],args(V)[1]));

In [ ]:
eigenvalues(dfdv);

In [ ]:
Ginv;

In [ ]:
Ginv : subst(diff(rho,T)=-rho/T,Ginv);

In [ ]:
dfdv : subst(c^2=gamma*p/rho,dfdv);

In [ ]:
dfdv : subst(p=rho*R*T,dfdv);

In [ ]:
Gdfdv : ratsimp(Ginv.dfdv);

In [ ]:
eigenvalues(dfdv);

We now need to construct the matrix $\Gamma^{-1} \frac{\partial F}{\partial V}$

Note that this is simply the matrix in front of the convective flux.

With $\frac{\partial F}{\partial V} = \frac{\partial F}{\partial U} \cdot \frac{\partial U}{\partial V}$


In [ ]:
dFdV : ratsimp(dFdU*dUdV);
dFdV : ratsimp(subst(consSystem,dFdV));
dFdV : ratsimp(subst(E=rho*e+rho*(u*u+v*v)/2,dFdV));
ratsimp(eigenvalues(dFdV));
ratsimp(eigenvalues(ratsimp(Ginv*dFdV)));

H=cpT+(u^2+v^2+w^2)/2; 

so:
$$\frac{dH}{du}=u $$
$$\frac{dH}{dv}=v $$
$$\frac{dH}{dw}=w $$
$$\frac{dH}{dT}=cp $$


weiss and smith eq. 2

In [ ]:
gradef(H,T,cp);
gradef(H,u,u);
gradef(H,v,v);
gradef(H,w,w);

In [ ]:
dWdQ : ratsimp(jacobian(args(W)[1],args(Q)[1]));

In [ ]:
G : ratsubst(1/b^2,diff(rho,p),dWdQ);

In [ ]:
G : ratsimp(ratsubst(cp*T+u^2/2,H,G));

In [ ]:
Ginv : ratsimp(invert(G));

In [ ]:
B : ratsimp(Ginv.A1.G);

In [ ]:
eigenvalues(B);

In [ ]:
/*remove(rho,dependency);*/
dUdV : ratsimp(jacobian(args(U)[1],args(V)[1]));

In [ ]:
M : dUdV;
Minv : ratsimp(invert(M));
A1;
Atilde : ratsimp(Minv.A1.M);
eigenvalues(Atilde);

In [ ]:
M1 : ratsimp(ratsubst(cp*T+u^2/2,H,M));

In [ ]:
M1inv : ratsimp(invert(M1));

In [ ]:
A : ratsimp(u,U,A);
Atilde : A;

weiss and smith Eq.3

In [ ]:
/*K:matrix([1,0,0,0,0],[-u,1,0,0,0],[-v,0,1,0,0],[-w,0,0,1,0],[-(H-(u^2+v^2+w^2)),-u,-v,-w,1]);*/
K:matrix([1,0,0],[-u,1,0],[-(H-(u^2)),-u,1]);

weiss and smith eq.4

In [ ]:
Gp:ratsimp(K.dWdQ);

weiss and smith, eq. 10

In [ ]:
G : ratsimp(invert(K).Gp);

In [ ]:
ratsimp(jacobian(args(Ftilde)[1],args(Q)[1]));